## <b><center>CUSTOMER TRANSACTIONS</b></center>

##### <b>IMPORT LIBRARY PACKAGES</b>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from datetime import datetime
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, classification_report

##### <b>LOAD & DISPLAY DATASET</b>

In [ ]:
customer_transactions = pd.read_csv("sample_dataset.csv")
customer_transactions

##### <b>DATA UNDERSTANDING</b>

The data consists of customer transaction records with the following columns:

* customer_id: Unique identifier for each customer.

* Name: First name of the customer.

* Surname: Last name of the customer.

* Gender: the sexual orientation of the customer.

* Birthdate: the day, month and year the customer was born.

* Transaction Amount: Amount of transaction made.
* Date: Date of the transaction.
* Merchant Name: the name of the merchant making the transactions.
* Category: Product category.

##### <b>DATA EXPLORATION</b>

CHECK THE SHAPE

In [ ]:
print('The number of rows and columns in this dataset is', customer_transactions.shape, 'respectively')

CHECK THE SIZE

In [ ]:
print('The size of this dataset is', customer_transactions.size) 

CHECK THE COLUMNS

In [ ]:
customer_transactions.columns

CHECK INFO

In [ ]:
customer_transactions.info()

SUMMARY STATISTICS

In [ ]:
customer_transactions.describe(include='all').T 

##### <b>DATA PREPARATION</b>

CHECK FOR MISSING VALUES

In [ ]:
customer_transactions.isna()

In [ ]:
customer_transactions.isna().sum()

DISPLAY HEATMAP FOR MISSING VALUES

In [ ]:
# Create a boolean DataFrame indicating missing values
missing_values = customer_transactions.isna()

# Plot the heatmap
fig = px.imshow(missing_values, aspect="auto", color_continuous_scale='Viridis', title='Missing Values Heatmap')

# Customize the axes and color scale
fig.update_layout(xaxis_title='Columns', yaxis_title='Index')
fig.update_coloraxes(colorbar_title='Missing')

# Show the plot
fig.show()

##### <b>DATA CLEANING</b>

In [ ]:
customer_transactions_copy = customer_transactions.copy

CHECK UNIQUE VALUES IN THE GENDER COLUMN

In [ ]:
customer_transactions['Gender'].unique()

CHECK THE VALUE COUNT OF EACH GENDER

In [ ]:
customer_transactions['Gender'].value_counts()

REPLACE THE INITIALS OF THE GENDER TO FULL NAME

In [ ]:
customer_transactions['Gender'] = customer_transactions['Gender'].replace('F', 'Female')
customer_transactions['Gender'] = customer_transactions['Gender'].replace('M', 'Male')

FILL NAN VALUES WITH UNKNOWN

In [ ]:
#Replace nan values in the Gender column with "Unknown"
customer_transactions['Gender'].fillna('Prefer not to say', inplace=True)

CHECK VALUE COUNTS AFTER THE CORRECTION AND REPLACEMENT

In [ ]:
customer_transactions['Gender'].value_counts()

In [ ]:
# Create a boolean DataFrame indicating missing values
missing_values = customer_transactions.isna()

# Plot the heatmap
fig = px.imshow(missing_values, aspect="auto", color_continuous_scale='Viridis', title='Missing Values Heatmap')

# Customize the axes and color scale
fig.update_layout(xaxis_title='Columns', yaxis_title='Index')
fig.update_coloraxes(colorbar_title='Missing')

# Show the plot
fig.show()

In [ ]:
customer_transactions.isnull().sum()

##### <b>EXPLORATION DATA ANALYSIS, (E.D.A) </b>

In [ ]:
customer_transactions.hist(figsize=(10,4))

UNIVARIATE ANALYSIS ON EACH COLUMN

CUSTOMER ID

In [ ]:
print("Number of unique Customer IDs:", customer_transactions['Customer ID'].nunique())

NAME

In [ ]:
print("Most common names:\n", customer_transactions['Name'].value_counts().head())

SURNAME

In [ ]:
print("Most common surnames:\n", customer_transactions['Surname'].value_counts().head())

GENDER

In [ ]:
# Calculate gender counts
gender_counts = customer_transactions['Gender'].value_counts()

# Create a bar plot using Plotly
fig = px.bar(gender_counts, x=gender_counts.index, y=gender_counts.values, title='Gender Distribution')
fig.update_layout(xaxis_title='Gender', yaxis_title='Count')
fig.show()

BIRTHDATE

In [ ]:
# Convert Birthdate to datetime and extract Birth Year
customer_transactions['Birthdate'] = pd.to_datetime(customer_transactions['Birthdate'])
customer_transactions['Birth Year'] = customer_transactions['Birthdate'].dt.year

# Create a histogram of Birth Year using Plotly
fig = px.histogram(customer_transactions, x='Birth Year', title='Birth Year Distribution')
fig.update_layout(xaxis_title='Birth Year', yaxis_title='Count')
fig.show()

TRANSACTION AMOUNT

In [ ]:
# Transaction Amount

# Create a histogram of Transaction Amount using Plotly Express
fig = px.histogram(customer_transactions, x='Transaction Amount', title='Transaction Amount Distribution')
fig.update_layout(xaxis_title='Transaction Amount', yaxis_title='Count')
fig.show()

##### <b>BUSINESS ANALYSIS QUESTION</b>

Which category made the highest sales during the period?

1. What is the distribution of transaction amounts across different genders?

In [ ]:
# Box plot of transaction amounts by gender
fig1 = px.histogram(customer_transactions, x='Gender', y='Transaction Amount', title='Transaction Amount by Gender')
fig1.show()

2. How does the transaction amount vary by category?

In [ ]:
# Create a box plot with color grading by category using Plotly Express
fig2 = px.box(customer_transactions, 
              x='Category', 
              y='Transaction Amount', 
              title='Transaction Amount by Category', 
              color='Category', 
              color_discrete_sequence=px.colors.qualitative.Vivid)

fig2.update_layout(xaxis_title='Category', yaxis_title='Transaction Amount')
fig2.show()

In [ ]:
# Box plot of transaction amounts by category
fig2 = px.box(customer_transactions, x='Category', y='Transaction Amount', title='Transaction Amount by Category')
fig2.update_layout(xaxis_title='Category', yaxis_title='Transaction Amount')
fig2.show()

3. Which merchants have the highest number of transactions?

In [ ]:
# Bar plot of the number of transactions by merchant
merchant_counts = customer_transactions['Merchant Name'].value_counts().nlargest(10).reset_index()
merchant_counts.columns = ['Merchant Name', 'Number of Transactions']

# Create a bar plot with color grading using Plotly Express
fig = px.bar(merchant_counts, 
             x='Merchant Name', 
             y='Number of Transactions', 
             title='Top 10 Merchants by Number of Transactions', 
             color='Number of Transactions', 
             color_continuous_scale='Viridis')

fig.update_layout(xaxis_title='Merchant Name', yaxis_title='Number of Transactions')
fig.show()

4. How do transaction amounts vary over time?

In [ ]:
# Convert 'Date' to datetime if it's not already
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])

# Line plot of transaction amounts over time
customer_transactions.set_index('Date', inplace=True)
monthly_transactions = customer_transactions['Transaction Amount'].resample('M').sum()
fig4 = px.line(monthly_transactions, title='Monthly Transaction Amounts Over Time')
fig4.update_layout(xaxis_title='Date', yaxis_title='Total Transaction Amount')
fig4.show()

5. What is the relationship between customer age and transaction amount?

In [ ]:
# Calculate customer age
customer_transactions['Birthdate'] = pd.to_datetime(customer_transactions['Birthdate'])
customer_transactions['Age'] = pd.to_datetime('today').year - customer_transactions['Birthdate'].dt.year

# Scatter plot of transaction amount by age
fig5 = px.scatter(customer_transactions, x='Age', y='Transaction Amount', title='Transaction Amount by Age')
fig5.update_layout(xaxis_title='Age', yaxis_title='Transaction Amount')
fig5.show()

MACHINE LEARNING

In [ ]:
# Encode categorical variables
label_encoders = {}
for column in ['Gender', 'Category', 'Merchant Name']:
    le = LabelEncoder()
    customer_transactions[column] = le.fit_transform(customer_transactions[column])
    label_encoders[column] = le

# Feature Engineering
# Convert 'Birthdate' to age
customer_transactions['Birthdate'] = pd.to_datetime(customer_transactions['Birthdate'])
customer_transactions['Age'] = pd.to_datetime('today').year - customer_transactions['Birthdate'].dt.year

# Ensure 'Date' is datetime
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])

# Extract date features
customer_transactions['Year'] = customer_transactions['Date'].dt.year
customer_transactions['Month'] = customer_transactions['Date'].dt.month
customer_transactions['Day'] = customer_transactions['Date'].dt.day

# Drop columns that won't be used
customer_transactions.drop(columns=['Customer ID', 'Name', 'Surname', 'Birthdate', 'Date'], inplace=True)

# Split the data into training and test sets
X = customer_transactions.drop(columns=['Transaction Amount'])
y = customer_transactions['Transaction Amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Plot actual vs. predicted transaction amounts
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
fig = px.scatter(results, x='Actual', y='Predicted', title='Actual vs. Predicted Transaction Amounts')
fig.show()

In [ ]:
# Date
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])
customer_transactions['Year'] = customer_transactions['Date'].dt.year
customer_transactions['Month'] = customer_transactions['Date'].dt.month
customer_transactions['Day'] = customer_transactions['Date'].dt.day

sns.countplot(x='Year', data=customer_transactions)
plt.title('Transactions per Year')
plt.show()

sns.countplot(x='Month', data=customer_transactions)
plt.title('Transactions per Month')
plt.show()

sns.countplot(x='Day', data=customer_transactions)
plt.title('Transactions per Day')
plt.show()

In [ ]:
# Merchant Name
print("Most common merchants:\n", customer_transactions['Merchant Name'].value_counts().head())

# Category
category_counts = customer_transactions['Category'].value_counts()
sns.barplot(x=category_counts.index, y=category_counts.values)
plt.xticks(rotation=90)
plt.title('Transaction Category Distribution')
plt.show()

BIVARIATE ANALYSIS

Gender vs. Transaction Amount:

In [ ]:
sns.boxplot(x='Gender', y='Transaction Amount', data=customer_transactions)
plt.title('Transaction Amount by Gender')
plt.show()

Birth Year vs. Transaction Amount:

In [ ]:
customer_transactions['Birthdate'] = pd.to_datetime(customer_transactions['Birthdate'])
customer_transactions['Birth Year'] = customer_transactions['Birthdate'].dt.year
sns.scatterplot(x='Birth Year', y='Transaction Amount', data=customer_transactions)
plt.title('Transaction Amount by Birth Year')
plt.show()

Date vs. Transaction Amount:

In [ ]:
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])
customer_transactions.set_index('Date', inplace=True)
customer_transactions['Transaction Amount'].resample('M').sum().plot()
plt.title('Monthly Transaction Amounts Over Time')
plt.xlabel('Date')
plt.ylabel('Total Transaction Amount')
plt.show()

Category vs. Transaction Amount:

In [ ]:
sns.boxplot(x='Category', y='Transaction Amount', data=customer_transactions)
plt.xticks(rotation=-45)
plt.title('Transaction Amount by Category')
plt.show()

Merchant Name vs. Transaction Amount:

In [ ]:
top_merchants = customer_transactions['Merchant Name'].value_counts().index[:10]  # Select top 10 merchants for better visualization
sns.boxplot(x='Merchant Name', y='Transaction Amount', data=customer_transactions[customer_transactions['Merchant Name'].isin(top_merchants)])
plt.xticks(rotation=90)
plt.title('Transaction Amount by Merchant')
plt.show()

Gender vs. Category:

In [ ]:
gender_category_counts = pd.crosstab(customer_transactions['Gender'], customer_transactions['Category'])
sns.heatmap(gender_category_counts, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Gender vs. Category')
plt.show()

Gender vs. Merchant Name:

In [ ]:
gender_merchant_counts = pd.crosstab(customer_transactions['Gender'], customer_transactions['Merchant Name'])
sns.heatmap(gender_merchant_counts, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Gender vs. Merchant Name')
plt.show()

In [ ]:
# Question 1: Gender distribution of customers
gender_counts = customer_transactions['Gender'].value_counts()
gender_distribution = gender_counts / gender_counts.sum() * 100
print("Gender Distribution:\n", gender_distribution)

# Question 2: Age group with the highest transaction amounts
customer_transactions['Birthdate'] = pd.to_datetime(customer_transactions['Birthdate'])
customer_transactions['Age'] = (pd.to_datetime('today') - customer_transactions['Birthdate']).dt.days // 365
customer_transactions['Age Group'] = pd.cut(customer_transactions['Age'], bins=[0, 18, 30, 45, 60, 100], labels=['0-18', '19-30', '31-45', '46-60', '60+'])
age_group_transaction_amounts = customer_transactions.groupby('Age Group')['Transaction Amount'].sum()
print("Transaction Amounts by Age Group:\n", age_group_transaction_amounts)

# Question 3: Seasonal trend in transaction amounts
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])
customer_transactions['Month'] = customer_transactions['Date'].dt.month
monthly_transactions = customer_transactions.groupby('Month')['Transaction Amount'].sum()
monthly_transactions.plot(kind='bar')
plt.title('Monthly Transaction Amounts')
plt.xlabel('Month')
plt.ylabel('Total Transaction Amount')
plt.show()

# Question 4: Most popular merchants
top_merchants = customer_transactions['Merchant Name'].value_counts().head(10)
print("Top Merchants:\n", top_merchants)

# Question 5: Most common transaction categories
top_categories = customer_transactions['Category'].value_counts().head(10)
print("Top Categories:\n", top_categories)

# Question 6: Transaction amounts by gender
gender_transaction_amounts = customer_transactions.groupby('Gender')['Transaction Amount'].mean()
print("Transaction Amounts by Gender:\n", gender_transaction_amounts)

# Question 7: Transaction amounts by merchant
merchant_transaction_amounts = customer_transactions.groupby('Merchant Name')['Transaction Amount'].mean().sort_values(ascending=False).head(10)
print("Transaction Amounts by Merchant:\n", merchant_transaction_amounts)

# Question 8: Categories preferred by different genders
gender_category_counts = pd.crosstab(customer_transactions['Gender'], customer_transactions['Category'])
print("Categories by Gender:\n", gender_category_counts)

# Visualize Categories preferred by different genders using Plotly
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
    z=gender_category_counts.values,
    x=gender_category_counts.columns,
    y=gender_category_counts.index,
    colorscale='YlGnBu',
    text=gender_category_counts.values,
    texttemplate="%{text}",
    textfont={"size":12}
))

fig.update_layout(
    title='Gender vs. Category',
    xaxis_nticks=36,
    yaxis_nticks=36,
    xaxis_title='Category',
    yaxis_title='Gender'
)

fig.show()

DISTRIBUTION OF TRANSACTION AMOUNTS

In [ ]:
# Plot the distribution of Transaction Amounts using Plotly Express
fig = px.histogram(customer_transactions, x='Transaction Amount', nbins=30, marginal='rug', title='Distribution of Transaction Amounts', labels={'Transaction Amount': 'Transaction Amount', 'count': 'Frequency'})

# Show the plot
fig.show()

TRANSACTIONS OVER TIME

In [ ]:
# Ensure the Date column is of datetime type
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])

# Set the Date column as the index
customer_transactions.set_index('Date', inplace=True)

# Resample the data by month and sum the transaction amounts
monthly_transactions = customer_transactions.resample('M')['Transaction Amount'].sum().reset_index()

# Create the Plotly Express line plot
fig = px.line(monthly_transactions, x='Date', y='Transaction Amount', title='Monthly Transaction Amounts', labels={'Date': 'Date', 'Transaction Amount': 'Total Transaction Amount'})

# Show the plot
fig.show()

# Reset index after plotting (if needed)
#customer_transactions.reset_index(inplace=True)

TRANSACTIONS BY GENDER

In [ ]:
# Assuming df is your DataFrame
# Group by Gender and sum the Transaction Amount
gender_transaction = customer_transactions.groupby('Gender')['Transaction Amount'].sum().reset_index()

# Create the Plotly Express bar plot
fig = px.bar(gender_transaction, x='Gender', y='Transaction Amount', title='Total Transaction Amount by Gender', labels={'Gender': 'Gender', 'Transaction Amount': 'Total Transaction Amount'}, color='Gender',
                   color_discrete_sequence=['#636EFA', '#EF553B', '#BE664C'])

# Show the plot
fig.show()

AGE ANALYSIS

In [ ]:
#To further aid our analysis, I will be creating a new column that will generate the age of the customer as at the time of this analysis.

customer_transactions['Birthdate'] = pd.to_datetime(customer_transactions['Birthdate'])
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Birthdate'])

customer_transactions['Age'] = customer_transactions['Birthdate'].apply(lambda x: (datetime.now() - x).days // 365)

In [ ]:
# Create the histogram with Plotly Express
fig = px.histogram(customer_transactions, x='Age', nbins=20, title='Distribution of Customer Ages', labels={'Age': 'Age', 'count': 'Frequency'})

# Add a KDE (Kernel Density Estimate) trace
fig.add_trace(go.Histogram(x=customer_transactions['Age'], histnorm='density', nbinsx=20, opacity=0.75))
fig.add_trace(go.Scatter(x=customer_transactions['Age'], y=pd.Series(customer_transactions['Age']).value_counts().sort_index() / len(customer_transactions['Age']), mode='lines', name='KDE'))

# Update layout for better visualization
fig.update_layout(
    xaxis_title='Age',
    yaxis_title='Frequency',
    showlegend=True
)

# Show the plot
fig.show()

In [ ]:
# Calculate age
#current_year = pd.Timestamp.now().year
#customer_transactions['Age'] = current_year - customer_transactions['Birthdate'].dt.year

# Plot distribution of ages
sns.histplot(customer_transactions['Age'], kde=True)
plt.title('Distribution of Customer Ages')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Analyze transaction amount by age group
age_bins = [0, 18, 30, 40, 50, 60, 70, 80, 90, 100]
customer_transactions['Age Group'] = pd.cut(customer_transactions['Age'], bins=age_bins)
age_transaction = customer_transactions.groupby('Age Group')['Transaction Amount'].sum().reset_index()
sns.barplot(x='Age Group', y='Transaction Amount', data=age_transaction)
plt.title('Total Transaction Amount by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Total Transaction Amount')
plt.show()

In [ ]:
# Plot total transaction amount by category
category_transaction = customer_transactions.groupby('Category')['Transaction Amount'].sum().reset_index()
sns.barplot(x='Category', y='Transaction Amount', data=category_transaction)
plt.title('Total Transaction Amount by Category')
plt.xlabel('Category')
plt.ylabel('Total Transaction Amount')
plt.xticks(rotation=90)
plt.show()

TRANSACTION BY CATEGORY

In [ ]:
category_transaction_amount = customer_transactions.groupby('Category')['Transaction Amount'].sum().sort_values(ascending=True)
print(category_transaction_amount)

In [ ]:
# Group by Category and sum the Transaction Amount
category_transaction = customer_transactions.groupby('Category')['Transaction Amount'].sum().reset_index()

# Sort the DataFrame by Transaction Amount in ascending order
category_transaction = category_transaction.sort_values(by='Transaction Amount', ascending=True)

# Create the Plotly Express bar plot with colorful bars
fig = px.bar(category_transaction, 
             x='Category', 
             y='Transaction Amount', 
             color='Category',  # Use the category to color the bars
             title='Total Transaction Amount by Category', 
             labels={'Category': 'Category', 'Transaction Amount': 'Total Transaction Amount'},
             color_discrete_sequence=px.colors.qualitative.Safe)  # Use a predefined color sequence

# Customize the plot for better readability
fig.update_layout(xaxis={'categoryorder':'total ascending'}, xaxis_tickangle=-90)

# Show the plot
fig.show()

In [ ]:
# Prepare data for clustering
clustering_data = customer_transactions.groupby('Customer ID').agg({
    'Transaction Amount': 'sum',
    'Age': 'mean'
}).reset_index()

# Standardize data
scaler = StandardScaler()
clustering_data_scaled = scaler.fit_transform(clustering_data[['Transaction Amount', 'Age']])

# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
clustering_data['Cluster'] = kmeans.fit_predict(clustering_data_scaled)

# Visualize clusters
sns.scatterplot(x='Transaction Amount', y='Age', hue='Cluster', data=clustering_data, palette='viridis')
plt.title('Customer Segmentation')
plt.xlabel('Total Transaction Amount')
plt.ylabel('Age')
plt.show()

In [ ]:
total_transaction_amount = customer_transactions['Transaction Amount'].sum()
average_transaction_amount = customer_transactions['Transaction Amount'].mean()
print(f"Total Transaction Amount: {total_transaction_amount}")
print(f"Average Transaction Amount: {average_transaction_amount}")

In [ ]:
customer_transactions.set_index('Date', inplace=True)
daily_transactions = customer_transactions['Transaction Amount'].resample('D').sum()
sns.lineplot(x=daily_transactions.index, y=daily_transactions.values)
plt.title('Daily Transaction Amount Over Time')
plt.show()

In [ ]:
top_merchants = customer_transactions.groupby('Merchant Name')['Transaction Amount'].sum().sort_values(ascending=False).head(10)
print(top_merchants)
top_merchants.plot(kind='bar')
plt.title('Top 10 Merchants by Transaction Amount')
plt.ylabel('Total Transaction Amount')
plt.show()

In [ ]:
# Plot distribution of Transaction Amounts
sns.histplot(customer_transactions['Transaction Amount'], kde=True)
plt.title('Distribution of Transaction Amounts')
plt.xlabel('Transaction Amount')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Create a histogram with KDE for Transaction Amounts
fig = px.histogram(customer_transactions, x='Transaction Amount', nbins=30, marginal='rug', 
    title='Distribution of Transaction Amounts', labels={'Transaction Amount': 'Transaction Amount'})

# Show the plot
fig.show()

FEATURE ENGINEERING

CREATE A NEW COLUMN FOR AGE

In [ ]:
#To further aid our analysis, I will be creating a new column that will generate the age of the customer as at the time of this analysis.

customer_transactions['Birthdate'] = pd.to_datetime(customer_transactions['Birthdate'])
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])

customer_transactions['Age'] = customer_transactions['Birthdate'].apply(lambda x: (datetime.now() - x).days // 365)

In [ ]:
customer_transactions

In [ ]:
# Calculate age
current_year = pd.Timestamp.now().year
customer_transactions['Age'] = current_year - customer_transactions['Birthdate'].dt.year

# Create histogram with KDE
fig = px.histogram(customer_transactions, x='Age', nbins=30, marginal='violin', title='Age Distribution')

# Show the plot
fig.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(customer_transactions['Transaction Amount'], bins=30, kde=True)
plt.title('Transaction Amount Distribution')

In [ ]:
plt.subplot(1, 2, 2)
sns.histplot(customer_transactions['Age'], bins=30, kde=True)
plt.title('Age Distribution')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.countplot(x='Gender', data=customer_transactions)
plt.title('Gender Distribution')


In [ ]:
plt.subplot(1, 2, 2)
sns.countplot(y='Category', data=customer_transactions)
plt.title('Transaction Categories')
plt.show()

In [ ]:
customer_transactions['YearMonth'] = customer_transactions['Date'].dt.to_period('M')

plt.figure(figsize=(12, 5))
sns.lineplot(data=customer_transactions.groupby('YearMonth')['Transaction Amount'].sum().reset_index(), x='YearMonth', y='Transaction Amount')
plt.title('Monthly Transaction Amount Over Time')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
sns.boxplot(x='Gender', y='Transaction Amount', data=customer_transactions)
plt.title('Transaction Amount by Gender')
plt.show()

customer_transactions['AgeGroup'] = pd.cut(customer_transactions['Age'], bins=[0, 18, 35, 50, 65, 100], labels=['<18', '18-35', '35-50', '50-65', '65+'])

plt.figure(figsize=(12, 5))
sns.boxplot(x='AgeGroup', y='Transaction Amount', data=customer_transactions)
plt.title('Transaction Amount by Age Group')
plt.show()

In [ ]:
top_merchants = customer_transactions.groupby('Merchant Name')['Transaction Amount'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(12, 5))
sns.barplot(x=top_merchants.values, y=top_merchants.index)
plt.title('Top 10 Merchants by Transaction Amount')
plt.show()

In [ ]:
category_spending = customer_transactions.groupby('Category')['Transaction Amount'].sum().sort_values(ascending=False)
plt.figure(figsize=(12, 5))
sns.barplot(x=category_spending.values, y=category_spending.index)
plt.title('Spending by Category')
plt.show()

In [ ]:
corr_matrix = customer_transactions[['Age', 'Transaction Amount']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

DATA PREPROCESSING

In [ ]:
customer_transactions['Gender'].unique()

In [ ]:
customer_transactions['Gender'] = customer_transactions['Gender'].replace ('F', 'Female')
customer_transactions['Gender'] = customer_transactions['Gender'].replace ('M', 'Male')
customer_transactions['Gender'] = customer_transactions['Gender'].replace ('nan', 'Prefer not to say')

In [ ]:
customer_transactions['Gender'].unique()

In [ ]:
customer_transactions['Transaction Amount'].unique()

In [ ]:
customer_transactions['Merchant Name'].unique()

UNIVARIATE ANALYSIS

In [ ]:
# Extract the values from the 'Gender' column and count occurrences
gender_counts = customer_transactions['Gender'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']

colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']

# Create the pie chart
fig = px.pie(gender_counts, names='Gender', values='Count', title='Gender Distribution')

# Update the chart to add data labels
fig.update_traces(textinfo='percent+label')

# Show the plot
fig.show()

In [ ]:
# Plot transactions over time
customer_transactions.set_index('Date', inplace=True)
customer_transactions.resample('M')['Transaction Amount'].sum().plot()
plt.title('Monthly Transaction Amounts')
plt.xlabel('Date')
plt.ylabel('Total Transaction Amount')
plt.show()

# Reset index after plotting
customer_transactions.reset_index(inplace=True)

In [ ]:
# Plot total transaction amount by gender
gender_transaction = customer_transactions.groupby('Gender')['Transaction Amount'].sum().reset_index()
sns.barplot(x='Gender', y='Transaction Amount', data=gender_transaction)
plt.title('Total Transaction Amount by Gender')
plt.xlabel('Gender')
plt.ylabel('Total Transaction Amount')
plt.show()

UNIVARIATE ANALYSIS

In [ ]:
# 1. Customer ID (int64)
print("Customer ID:")
print("Count of Unique Values:", customer_transactions['Customer ID'].nunique())
print("Summary Statistics:")
print(customer_transactions['Customer ID'].describe())

# 2. Name (object)
print("\nName:")
print("Count of Unique Names:", customer_transactions['Name'].nunique())
print("Most Common Names:")
print(customer_transactions['Name'].value_counts().head())
customer_transactions['Name_Length'] = customer_transactions['Name'].apply(len)
print("Mean Length:", customer_transactions['Name_Length'].mean())
print("Median Length:", customer_transactions['Name_Length'].median())
print("Missing Values:", customer_transactions['Name'].isnull().sum())

# 3. Surname (object)
print("\nSurname:")
print("Count of Unique Surnames:", customer_transactions['Surname'].nunique())
print("Most Common Surnames:")
print(customer_transactions['Surname'].value_counts().head())
customer_transactions['Surname_Length'] = customer_transactions['Surname'].apply(len)
print("Mean Length:", customer_transactions['Surname_Length'].mean())
print("Median Length:", customer_transactions['Surname_Length'].median())
print("Missing Values:", customer_transactions['Surname'].isnull().sum())

# 4. Gender (object)
print("\nGender:")
print("Count of Each Gender:")
print(customer_transactions['Gender'].value_counts())
print("Percentage Distribution:")
print(customer_transactions['Gender'].value_counts(normalize=True) * 100)
print("Missing Values:", customer_transactions['Gender'].isnull().sum())

# 5. Birthdate (object)
print("\nBirthdate:")
customer_transactions['Birth_Year'] = pd.to_datetime(customer_transactions['Birthdate']).dt.year
print("Distribution of Birth Years:")
print(customer_transactions['Birth_Year'].value_counts().sort_index())
print("Distribution of Birth Months:")
print(pd.to_datetime(customer_transactions['Birthdate']).dt.month.value_counts().sort_index())
customer_transactions['Age'] = pd.to_datetime('now').year - pd.to_datetime(customer_transactions['Birthdate']).dt.year
print("Mean Age:", customer_transactions['Age'].mean())
print("Median Age:", customer_transactions['Age'].median())
print("Youngest:", customer_transactions['Age'].min())
print("Oldest:", customer_transactions['Age'].max())
print("Missing Values:", customer_transactions['Birthdate'].isnull().sum())

# 6. Transaction Amount (float64)
print("\nTransaction Amount:")
print("Summary Statistics:")
print(customer_transactions['Transaction Amount'].describe())
plt.hist(customer_transactions['Transaction Amount'], bins=20)
plt.xlabel('Transaction Amount')
plt.ylabel('Frequency')
plt.title('Distribution of Transaction Amounts')
plt.show()
# Outlier detection (can be adjusted as needed)
outliers = customer_transactions[(customer_transactions['Transaction Amount'] - customer_transactions['Transaction Amount'].mean()).abs() > 3 * customer_transactions['Transaction Amount'].std()]
print("Number of Outliers:", len(outliers))

# 7. Date (object)
print("\nDate:")
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])
print("Distribution of Transactions Over Time:")
print(customer_transactions['Date'].dt.to_period('M').value_counts().sort_index())
print("Missing Values:", customer_transactions['Date'].isnull().sum())

# 8. Merchant Name (object)
print("\nMerchant Name:")
print("Count of Unique Merchant Names:", customer_transactions['Merchant Name'].nunique())
print("Most Common Merchant Names:")
print(customer_transactions['Merchant Name'].value_counts().head())
print("Missing Values:", customer_transactions['Merchant Name'].isnull().sum())

# 9. Category (object)
print("\nCategory:")
print("Count of Unique Categories:", customer_transactions['Category'].nunique())
print("Most Common Categories:")
print(customer_transactions['Category'].value_counts().head())
print("Missing Values:", customer_transactions['Category'].isnull().sum())

In [ ]:
# 4. Gender
print("\nGender:")
print("- Count of Each Gender:")
print(customer_transactions['Gender'].value_counts())
print("- Percentage Distribution:")
print(customer_transactions['Gender'].value_counts(normalize=True) * 100)
print("- Missing Values:", customer_transactions['Gender'].isnull().sum())

In [ ]:
# Columns to exclude from the loop
#exclude_columns = ['latitude', 'longitude', 'Pin code']

# Loop through each column
for column in customer_transactions.columns:
    # Skip excluded columns
   # if column in exclude_columns:
   #     continue
    
    # Get unique values and their counts
    value_counts = customer_transactions[column].value_counts()
    
    # Create bar chart
    fig = px.bar(value_counts, x=value_counts.index, y=value_counts.values, 
                 labels={'x': column, 'y': 'Count'}, 
                 title=f"Value Counts for {column}")
    
    # Show plot
    fig.show()

#### TASK 1 - CHECK FOR DUPLICATED ROWS

In [ ]:
Total_dup = customer_transactions.duplicated().sum()
print('The total number of duplicate rows in this dataset is', Total_dup)

#### TASK 2 - CHECK FOR MISSING VALUES

In [ ]:
customer_transactions.isna().sum()

In [ ]:
Total_MV = customer_transactions.isna().sum().sum()
print('The total number of missing values in this dataset is', Total_MV)

In [ ]:
#Check for the percentage for the missing values.

missing_gender = customer_transactions['Gender'].isnull().sum()
total_rows = len(customer_transactions)
missing_percentage = (missing_gender / total_rows) * 100

print(f"Missing Gender Values: {missing_gender} ({missing_percentage:.2f}%)")

#### HEATMAP FOR MISSING VALUES

In [ ]:
sns.heatmap(customer_transactions.isnull())

#### INSIGHTS

The results showed that there are 5,047 missing values in the gender column which sums up to 10.09% of the dataset. 

In dealing with this, I will replace all the missing values in the gender column to UNDEFINED.

This presupposes that the these customers did not find their gender orientation listed in there to select, hence the empty field.

#### QUESTIONS

 - Which category made the highest sales during the period?
 - Which age grouping patronized more of cosmetics, clothings and travel?
 - Which date had the highest patronage for Clothing?

#### Question 1
 - Which category made the highest sales during the period?

In [ ]:
# Group the data by category and calculate total sales for each category
category_sales = customer_transactions.groupby('Category')['Transaction Amt ($)'].sum()

# Print the total sales for each category
print(category_sales)

In [ ]:
customer_transactions = category_sales.reset_index()
customer_transactions.columns = ['Category', 'Transaction Amt ($)']

# Plotting the bar chart using Plotly Express
fig = px.bar(customer_transactions, x='Category', y='Transaction Amt ($)',
             title='Total Sales by Category',
             labels={'Transaction Amt ($)': 'Transaction Amt ($)'},
             text='Transaction Amt ($)',
             width=800, height=500)

# Rotate x-axis labels for better readability
fig.update_xaxes(tickangle=45)

# Show the plot
fig.show()

In [ ]:
# Find the category with the highest total sales
highest_sales_category = category_sales.idxmax()
highest_sales_amount = category_sales.max()

print(f"The category with the highest sales is '{highest_sales_category}' with a total sales amount of $ {highest_sales_amount:.2f}")

#### Question 2
 - Which age group patronized more of cosmetics, clothings and travel?

In [133]:
# Convert the 'Date of Birth' column to datetime
customer_transactions['Date of Birth'] = pd.to_datetime(customer_transactions['Date of Birth'])

# Calculate age based on the current date
current_date = datetime.now()
customer_transactions['Age'] = (current_date - customer_transactions['Date of Birth']) // pd.Timedelta(days=365)

customer_transactions

KeyError: 'Date of Birth'

In [132]:
# Group Ages into categories: young, middle age, and old

# Define age bins and labels
age_bins = [0, 30, 50, 100]  # You can adjust these age boundaries as needed

age_labels = ['Young', 'Middle Age', 'Old']

# Apply age grouping
customer_transactions['Age Group'] = pd.cut(customer_transactions['Age'], bins=age_bins, labels=age_labels, right=False)


KeyError: 'Age'

In [ ]:
customer_transactions.head()

In [ ]:
# Create a pie chart to visualize the results
labels = ['Cosmetic', 'Clothing', 'Travel']
sizes = [grouped_data['Travel'].max(), grouped_data['Clothing'].max(), grouped_data['Cosmetic'].max()]
explode = (0.1, 0, 0)  # Explode the 1st slice (Cosmetic)

plt.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title('Age Groups Patronizing Products')

plt.show()

#### Question 3
 - Which date had the highest patronage for Clothing?

In [ ]:
# List of categories you want to analyze
categories_to_analyze = ['Cosmetic', 'Clothing', 'Travel' , 'Electronics' , 'Market', 'Restaurant']

# Iterate through the categories
for category in categories_to_analyze:
    # Filter the data for the specific category
    category_data = customer_transactions[customer_transactions['Category'] == category]
    
    # Group the data by date and count the number of purchases for each date
    date_patronage = category_data.groupby('Date').size()
    
    # Find the date with the highest patronage for the category
    date_with_highest_patronage = date_patronage.idxmax()
    highest_patronage_count = date_patronage.max()
    
    print(f"For the '{category}' category:")
    print(f"The date with the highest patronage is {date_with_highest_patronage} with {highest_patronage_count} purchases.")
    print()


 #### Question 4
 - What is the total number of person based on their gender patronized restaurant?

In [ ]:
# Filter the data for the "Restaurant" category
restaurant_data = customer_transactions[customer_transactions['Category'] == 'Restaurant']

# Group the data by gender and count the number of patrons for each gender
gender_patrons = restaurant_data['Gender'].value_counts()

print("Total number of people who patronized the restaurant based on gender:")
print(gender_patrons)

In [ ]:
customer_transactions = pd.DataFrame({'Gender': gender_patrons.index, 'Number of Patrons': gender_patrons.values})

# Creating a bar chart using Plotly Express
fig = px.bar(customer_transactions, x='Gender', y='Number of Patrons',
             color='Gender',
             title='Total Number of Restaurant Patrons by Gender',
             labels={'Number of Patrons': 'Number of Patrons', 'Gender': 'Gender'},
             width=1000, height=00)

# Show the plot
fig.show()

 #### Question 5
 - What is the trend in the transactions?

In [ ]:
# Convert the 'transaction_date' column to a datetime object
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])

# Group the data by transaction date and count the number of transactions for each date
transaction_trend = customer_transactions.groupby('Date').size()

# Create a line chart to visualize the transaction trend
plt.figure(figsize=(12, 6))
plt.plot(transaction_trend.index, transaction_trend.values, marker='o', color='red')
plt.title('Transaction Trend Over The Transaction Period.')
plt.xlabel('Transaction Date')
plt.ylabel('Number of Transactions')
plt.grid(True)

# Optionally, format the x-axis labels for date
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.DayLocator(interval=10))  # Adjust the interval as needed

plt.xticks(rotation=45)

plt.show()

 #### Question 6
 - What is the percentage for the various gender? 

In [ ]:
#value counts for each gender

customer_transactions['Gender'].value_counts()

In [ ]:
# Assuming your DataFrame has a 'Gender' column, you can count the number of transactions for each gender
gender_counts = customer_transactions['Gender'].value_counts()

# Create a pie chart
plt.figure(figsize=(7, 7))
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Gender Distribution Percentage')

# Add a legend
plt.legend()

plt.show()

In [ ]:
# Group by "Age Group" and count the number of each gender within each age group
age_gender_distribution = customer_transactions.groupby(['Age Group', 'Gender']).size().unstack(fill_value=0)

# Display the age-gender distribution
print(age_gender_distribution)

In [ ]:
# Plot the age-gender distribution as a bar chart
age_gender_distribution.plot(kind='bar')
plt.title('Age-Gender Distribution')
plt.xlabel('Age Group')
plt.ylabel('Count')

plt.show()

 #### Question 7
 - Which merchant name had the highest number of transactions during the period?

In [ ]:
# Group the data by the merchant and count the number of transactions for each merchant

merchant_transaction_counts = customer_transactions['Merchant Name'].value_counts()

merchant_with_highest_transactions = merchant_transaction_counts.idxmax()

highest_transaction_count = merchant_transaction_counts.max()

print(f"The merchant with the highest number of transactions is '{merchant_with_highest_transactions}'")
print()
print(f"They have {highest_transaction_count} transactions.")